In [1]:
import tqdm 
import pandas as pd
import akshare as ak

def filter_shsz(v):
    code,name = v["code"],v["name"]
    if code.startswith("68") or code.startswith("8")or code.startswith("4"):
        return False
    elif "ST" in name or "退市" in name or "C" in name:
        return False
    elif code[0] not in ("0","3","6"):
        return False
    return True

def reform_shsz(code):
    if code.startswith("3") or code.startswith("00"):
        return "sz"+code
    elif code.startswith("6"):
        return "sh"+code
    return code
    
    
    
df = ak.stock_zh_a_spot_em()
df = df.rename(columns={"代码":"code","名称":"name"})
df = df[df.apply(filter_shsz,axis=1)]
df["code"] = df["code"].apply(reform_shsz)
cdf = df[["code","name"]]
lst = []
hdf = pd.read_csv("stock_hist.csv")
hs = set(hdf["code"].drop_duplicates())
cdf = cdf[~cdf.code.isin(hs)]
lst = [hdf]
bar = tqdm.tqdm(total =cdf.shape[0])

for i,(idx,v)in enumerate(cdf.iterrows()):
    bar.update()
    if i>1000:
        break
    try:
        tdf = ak.stock_zh_a_cdr_daily(symbol=v["code"], start_date="20241116")[["date","open","high","low","close","volume","amount"]]
        tdf[["code","name"]]=(v["code"],v["name"])
        lst.append(tdf)
    except:
        print(v)
if len(lst)>1:
    hdf = pd.concat(lst)
    hdf['code'] = hdf['code'].astype(str)
    hdf.to_csv("stock_hist.csv",index=False)


/Users/john/anaconda3/lib/python3.11/site-packages/akshare/__init__.py:2977: UserWarning: 为了支持更多特性，请将 Pandas 升级到 2.2.0 及以上版本！
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: 'stock_hist.csv'

In [4]:
import akshare as ak
import datetime

def filter_shsz(v):
    code,name = v["code"],v["name"]
    if code.startswith("68") or code.startswith("8")or code.startswith("4"):
        return False
    elif "ST" in name or "退市" in name or "C" in name:
        return False
    elif code[0] not in ("0","3","6"):
        return False
    return True


def reform_shsz(code):
    if code.startswith("3") or code.startswith("00"):
        return "sz"+code
    elif code.startswith("6"):
        return "sh"+code
    return code
    
today = datetime.datetime.now()
cdf = ak.stock_zh_a_spot_em()
cdf = cdf.rename(columns={"代码":"code","名称":"name"})
cdf = cdf[cdf.apply(filter_shsz,axis=1)]
cdf["code"] = cdf["code"].apply(reform_shsz)
cdf = cdf.rename(columns={"今开":"open","最低":"low","最高":"high","最新价":"close",
                         "成交量":"volume","成交额":"amount"})
cdf["date"]=today.strftime('%Y-%m-%d')
cdf["volume"] = cdf["volume"]*100
cdf["amount"] = cdf["amount"]
cdf = cdf[["date","code","name","open","low","high","close","volume","amount"]]
# hdf = pd.read_csv("stock_hist.csv")
# df = pd.concat([hdf,cdf])
# cdf.to_csv("stock_today.csv",index=False)
cdf.to_csv("stock_today.csv",index=False)
cdf.head()

,date,code,name,open,low,high,close,volume,amount
0,2024-12-30,sz001391,N国货航,10.00,9.11,12.50,9.30,613161700.0,6.210294e+09
1,2024-12-30,sz300933,中辰股份,7.10,7.08,8.47,8.47,107237500.0,8.890926e+08
2,2024-12-30,sz300067,安诺其,5.78,5.68,6.19,6.19,158813500.0,9.611870e+08
3,2024-12-30,sz301600,慧翰股份,170.43,170.26,199.88,199.26,4538500.0,8.605914e+08
4,2024-12-30,sz301315,威士顿,65.00,63.88,78.98,76.78,24607000.0,1.725814e+09


In [2]:
import pandas as pd
import tqdm
today = datetime.datetime.now()
bias = 0
cdate, pdate = today+datetime.timedelta(days=-bias),today+datetime.timedelta(days=-bias-1)
cdate,pdate = cdate.strftime('%Y-%m-%d'),pdate.strftime('%Y-%m-%d')
df = pd.read_csv(f"daily/stock_{pdate}.csv")
cdf = pd.read_csv("stock_today.csv")
df = pd.concat([df,cdf])
df.to_csv(f"daily/stock_{cdate}.csv",index=False)
cdf.to_csv(f"daily/stock_daily_{cdate}.csv",index=False)